In [1]:
import torch
import torch.nn as nn
import torchvision
from torchinfo import summary
from torchvision import transforms 
from torch.utils.data import DataLoader

In [4]:
transform_train = torchvision.transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])


In [5]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform_train)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True,
                                            transform=transform_test)


In [15]:
classes = train_dataset.classes
print(classes)


['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [16]:
img, label = train_dataset[0]
print(img.shape, label)

torch.Size([3, 224, 224]) 6


In [17]:
device = 'cpu'
if hasattr(torch,'mps') and torch.backends.mps.is_available():
    device = 'mps'
    print("MPS is available")

MPS is available


In [18]:
from torchvision import models
import torch.nn as nn

vgg16 = models.vgg16(pretrained=True)

for param in vgg16.features.parameters():
    param.requires_grad = False

x = torch.randn(1, 3, 224, 224)
out = vgg16.features(x)
print(out.shape)  # Use this to set the first Linear layer

torch.Size([1, 512, 7, 7])


In [19]:
import torchvision.models as models

class CustomVGG16(nn.Module):

    def __init__(self, num_classes=10, dropout=0.5, freeze_features=True):

        super(CustomVGG16, self).__init__()

        self.vgg16 = models.vgg16(pretrained=True)

        if freeze_features:
            for param in self.vgg16.features.parameters():
                param.requires_grad = False

        self.vgg16.classifier = nn.Sequential(
            nn.Linear(512*7*7, 256), 
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        
        return self.vgg16(x)


In [20]:
from tqdm import tqdm

In [21]:

def objective(trial):


    # Hyperparameters to tune

    dropout_rate = trial.suggest_uniform('dropout_rate', 0.2, 0.5)
    
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)

    optimizer_name = trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'RMSprop'])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    num_epochs = trial.suggest_int('num_epochs', 10, 30)

    num_channels = 3
    num_classes = 10

    model = CustomVGG16(num_classes,dropout_rate).to(device)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           shuffle=True)

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                          shuffle=False)

    # Optimizer
    if optimizer_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == 'RMSprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(num_epochs):
        
        model.train()

        train_loader_tqdm = tqdm(train_loader, desc=f"Trial {trial.number} | Epoch {epoch+1}/{num_epochs}")

        for batch_features, batch_labels in train_loader:

            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(batch_features)

            loss = criterion(outputs, batch_labels)
            
            loss.backward()
            
            optimizer.step()

            train_loader_tqdm.set_postfix(loss=loss.item())


    # Validation loop
    model.eval()  # how the model layers behave like disable dropout and uses running mean and variance.

    correct = 0
    total = 0
    
    with torch.no_grad():

        for batch_features, batch_labels in test_loader:
        
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        
            outputs = model(batch_features)
            _, predicted = torch.max(outputs, 1)
        
            total += batch_labels.size()[0]
            correct += (predicted == batch_labels).sum().item()
            

    accuracy = correct / total
    return accuracy

In [22]:
import optuna
pruner = optuna.pruners.MedianPruner() 

study = optuna.create_study(direction='maximize', pruner=pruner)
study.optimize(objective, n_trials=5) 


[I 2025-12-07 22:03:03,719] A new study created in memory with name: no-name-bed5e18a-ee30-4be5-91f9-d48985548ac1
/var/folders/68/g_j6jm596l5dg7k28vc763fh0000gn/T/ipykernel_8826/934188659.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.2, 0.5)
/var/folders/68/g_j6jm596l5dg7k28vc763fh0000gn/T/ipykernel_8826/934188659.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)
/var/folders/68/g_j6jm596l5dg7k28vc763fh0000gn/T/ipykernel_8826/934188659.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://gi

KeyboardInterrupt: 

In [ ]:
print("Best hyperparameters:", study.best_params)
print("Best accuracy:", study.best_value)